In [1]:
import torchvision.models as models
import torch
import numpy as np
import paddle
from config_cvt.default import get_config
from cvt import generate_model



In [2]:

config=get_config('./config_cvt/config.yaml')
paddle_model = generate_model(config)
paddle_model.eval()

torch_model= models.squeezenet1_1(pretrained=False)
torch_model.load_state_dict= torch.load('CvT-13-224x224-IN-1k.pth',map_location=torch.device('cpu'))


x = np.random.randn(2, 3, 224, 224).astype('float32')
x_paddle = paddle.to_tensor(x)
x_torch = torch.Tensor(x).to('cpu')

out_torch = torch_model(x_torch)
out_paddle = paddle_model(x_paddle)


merging config from ./config_cvt/config.yaml


In [3]:
out_torch = out_torch.data.cpu().numpy()
out_paddle = out_paddle.cpu().numpy()


In [4]:
out_torch

array([[0.05426907, 0.13646013, 0.18642074, ..., 0.6664542 , 0.02691144,
        0.21427536],
       [0.06572936, 0.15683094, 0.16821945, ..., 0.69633234, 0.0038871 ,
        0.15951963]], dtype=float32)

In [5]:
out_paddle

array([[-0.08503254,  0.02901448, -0.47339678, ...,  0.03890228,
        -0.17869541, -0.4637831 ],
       [-0.03906162,  0.00990262, -0.44092804, ..., -0.02356182,
        -0.15811093, -0.46786672]], dtype=float32)